# 케라스와 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기

In [1]:
import numpy as np
import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

You should consider upgrading via the '/Users/user/anaconda3/bin/python3 -m pip install --upgrade pip' command.
ERROR: tensorflow-metadata 0.23.0 has requirement absl-py<0.9,>=0.7, but you'll have absl-py 0.9.0 which is incompatible.
You should consider upgrading via the '/Users/user/anaconda3/bin/python3 -m pip install --upgrade pip' command.
버전:  2.0.0
즉시 실행 모드:  True
허브 버전:  0.8.0
GPU NOT AVAILABLE


## Load dataset

In [4]:
train_data, valid_data, test_data = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True
)

In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(3)))
train_examples_batch

<tf.Tensor: id=383, shape=(3,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion 

In [9]:
train_labels_batch

<tf.Tensor: id=384, shape=(3,), dtype=int64, numpy=array([0, 0, 0])>

## Define model

pre-trained text embedding을 사용

In [10]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=564, shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

## Train

In [15]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=valid_data.batch(512),
    verbose=1
)

Epoch 1/20
30/30 [==============================] - 3s 102ms/step - loss: 0.6955 - accuracy: 0.5432 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 2s 76ms/step - loss: 0.6277 - accuracy: 0.6158 - val_loss: 0.6036 - val_accuracy: 0.6421
Epoch 3/20
30/30 [==============================] - 2s 74ms/step - loss: 0.5861 - accuracy: 0.6634 - val_loss: 0.5698 - val_accuracy: 0.6787
Epoch 4/20
30/30 [==============================] - 2s 75ms/step - loss: 0.5474 - accuracy: 0.7021 - val_loss: 0.5359 - val_accuracy: 0.7166
Epoch 5/20
30/30 [==============================] - 2s 79ms/step - loss: 0.5096 - accuracy: 0.7395 - val_loss: 0.4983 - val_accuracy: 0.7467
Epoch 6/20
30/30 [==============================] - 2s 77ms/step - loss: 0.4673 - accuracy: 0.7773 - val_loss: 0.4601 - val_accuracy: 0.7748
Epoch 7/20
30/30 [==============================] - 2s 79ms/step - loss: 0.4231 - accuracy: 0.8045 - val_loss: 0.4284 - val_accuracy: 0.8016
Epoc

## Evaluation

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.matrics_names, results):
    print()